# Réarrengement data Solordinario

dict_keys(['file_name', 'env', 'vel', 'pitch', 'corde'])

In [1]:
import numpy as np
import glob 
import os.path
import scipy.io.wavfile as wavfile
import matplotlib.py

In [2]:
path = 'data'

def listdirectory(path): 
    fichier=[] 
    l = glob.glob(path+'\\*') 
    for i in l: 
        if os.path.isdir(i): fichier.extend(listdirectory(i)) 
        else: fichier.append(i) 
    return fichier

files = listdirectory(path)

In [3]:
long = 3 #durée à conserver en sec
for file in files:
    Fe, sig = wavfile.read(file)
    new_dir = 'data_2/'+file[5:]
    wavfile.write(new_dir, Fe, sig[:long*Fe])

# Dictionnaire avec Pitch et Velocite

In [4]:
import descriptors
import torch
import numpy as np

path = 'data_2'
files = listdirectory(path)
nb_files = len(files)
print(nb_files)
sample_per_sec = 250

dict_env = {'file_name' : [[]]*nb_files, 'env' : [[]]*nb_files, 'vel' : [[]]*nb_files, 'pitch' : [[]]*nb_files, 'corde' : [[]]*nb_files}
f = 0

for file in files:
    name = file[7:]
    vel = file[14:16]
    corde = file[17:19]
    if file[-6]=='#':
        pitch = file[-7:-4]
    else :
        pitch = file[-6:-4]
        
    Fe, sig = wavfile.read(file)
    sig = sig.astype(np.float32)
    env = np.zeros([2,long*sample_per_sec])
    env[0,:] = descriptors.extract_loudness(sig, sampling_rate=Fe, block_size=int(Fe/250), n_fft=2048)[:long*sample_per_sec]
    env[1,:] = descriptors.extract_pitch(sig, sampling_rate=Fe, block_size=int(Fe/250))[:long*sample_per_sec]
    
    dict_env['file_name'][f] = file
    dict_env['env'][f] = env
    dict_env['vel'][f] = vel
    dict_env['pitch'][f] = pitch
    dict_env['corde'][f] = corde
    print(f)
    f += 1
  

# Save
np.save('data_dict.npy', dict_env) 

# Load
# dict_r = np.load('data_dict.npy').item() 

# Création dataset pour torch (pas encore fait, les codes ici sont des copiés/collés, mais je n'ai pas encore travaillé dessus)

In [5]:
import os
import descriptors
import torch
import numpy as np

dict_r = np.load('data_dict.npy', allow_pickle=True).item()

#train_db = np.zeros([len(dict_r['env']),2,750])
#i = 0
#for sample in dict_r['env']:
#    train_db[i,:,:] = sample
#    i+=1
    
#train_dataset =  torch.utils.data.TensorDataset(torch.from_numpy(train_db))
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=25, shuffle=True)

In [6]:
dict_r.keys()

dict_keys(['file_name', 'env', 'vel', 'pitch', 'corde'])

In [13]:
loudEnv=dict_r['env'][0][0]
pitchEnv=dict_r["env"][0][1]
fileName=dict_r["file_name"][0]
pitch=dict_r["pitch"][0]
corde=dict_r["corde"][0]

In [14]:
plt.figure()
plt.plot(loudEnv)

NameError: name 'plt' is not defined